In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import os
from tqdm import tqdm
import time
import numpy as np
import torch.utils.data as data
from skimage import io, transform, img_as_float
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import random

In [2]:
Image_size = 32
num_classes = 2
BATCH_SIZE = 100
learning_rate = 0.001
transformation = transforms.Compose([
    transforms.ToTensor()
])

In [3]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, number_images, root_dir, folder, label_file , transforma=None):
        # Number of images to be loaded.
        self.number_images = number_images
        with open(root_dir+label_file, 'r') as file:
            lines=file.read().splitlines()
        self.Names_labels = []
        for element in lines:
            self.Names_labels.append(element.split(' '))
        
        # Here I construct the path to the images
        self.root_dir = root_dir+folder
        # this is to collect the transform, if I supply any.
        self.transforma = transforma

    def __len__(self):
        return self.number_images

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = os.path.join(self.root_dir,self.Names_labels[idx][0].split('/')[-1])
        image = io.imread(img_name)
        resized_img = transform.resize(image, (28, 28))
        sample = img_as_float(resized_img)
        target = int(float(self.Names_labels[idx][1]))

        if self.transforma:
            sample = self.transforma(sample)
        
        return [sample.float(), target]

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
Path_train ="."
number_images_training = 6966 
train_dataset = Dataset(number_images=number_images_training, root_dir=Path_train, folder = '/train',
                        label_file = '/COWC_train_list_detection.txt', transforma = transformation)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
Path_test = "."
number_images_testing = 2050
test_dataset = Dataset(number_images=number_images_testing, root_dir=Path_test, folder= '/test',
                       label_file = '/COWC_test_list_detection.txt', transforma = transformation)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Definition of the Normal Logistic Regression

In [6]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(2352 , num_classes)
        
    def forward(self, x):
        y_hat = self.linear(x)
        return y_hat

In [7]:
model = LogisticRegression(2352, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(7):
    loss_total = 0
    for i, (images, labels) in enumerate(train_dataloader):  
        images = images.view(images.size(0), -1).to(device)
        labels = Variable(labels).to(device)
        optimizer.zero_grad()  
        outputs = model(images)
        loss = criterion(outputs, labels)  
        loss.backward() 
        optimizer.step()
        loss_total = loss_total + loss.item()
    print(loss_total)

46.875478982925415
46.26075518131256
45.87075090408325
45.60809004306793
45.37194103002548
45.2206848859787
45.11464446783066


In [10]:
# Test the Model
correct = 0
total = 0
for images, labels in test_dataloader:
    labels = labels.to(device)
    images = images.view(images.size(0), -1).to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

In [11]:
print('Accuracy: {}%'.format(100 * correct / total))

Accuracy: 48.09756088256836%


### Definition of the Logistic regression with a non-linear activation function:

In [13]:
class LogisticRegression_Sigmoid(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.linear = nn.Linear(2352 , num_classes)
        
    def forward(self, x):
        y_hat = F.sigmoid(self.linear(x))
        return y_hat

In [16]:
model = LogisticRegression_Sigmoid(2352, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(7):
    loss_total = 0
    for i, (images, labels) in enumerate(train_dataloader):  
        images = images.view(images.size(0), -1).to(device)
        labels = Variable(labels).to(device)
        optimizer.zero_grad()  
        outputs = model(images)
        loss = criterion(outputs, labels)  
        loss.backward() 
        optimizer.step()
        loss_total = loss_total + loss.item()
    print(loss_total)

C:\Users\Utilisateur\anaconda3\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


47.46171897649765
46.81635820865631
46.7336688041687
46.67952758073807
46.67915105819702
46.6350993514061
46.601563811302185


In [17]:
# Test the Model
correct = 0
total = 0
for images, labels in test_dataloader:
    labels = labels.to(device)
    images = images.view(images.size(0), -1).to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

In [18]:
print('Accuracy: {}%'.format(100 * correct / total))

Accuracy: 47.17073059082031%
